In [1]:
import random
import heapq
from collections import deque

In [2]:
class Event:
    def __init__(self, time, event_type, queue_id, customer_id=None):
        self.time = time
        self.event_type = event_type
        self.queue_id = queue_id
        self.customer_id = customer_id

    def __lt__(self, other):
        return self.time < other.time

class Customer:
    def __init__(self, id):
        self.id = id

class Queue:
    def __init__(self, id, num_servers, capacity, service_time_range):
        self.id = id
        self.num_servers = num_servers
        self.capacity = capacity
        self.service_time_range = service_time_range  # (min_service_time, max_service_time)
        self.num_in_queue = 0  # Number of customers in the queue including those in service
        self.num_in_service = 0  # Number of customers being served
        self.state_times = {}  # key: state (number of customers), value: total time in that state
        self.last_event_time = 1.5  # Time of the last event that changed the state
        self.lost_customers = 0  # Number of lost customers
        self.waiting_line = deque()  # Queue of waiting customers

class Simulation:
    def __init__(self):
        self.event_list = []
        self.time = 1.5  # Start time
        self.random_numbers_used = 0
        self.customers = {}
        self.customer_id_counter = 0
        # Define the queues
        self.queues = {}
        # Initialize queues
        # Queue 1: G/G/2/3, arrivals between 1..4, service between 3..4
        self.queues[1] = Queue(id=1, num_servers=2, capacity=3, service_time_range=(3,4))
        # Queue 2: G/G/1/5, service between 2..3
        self.queues[2] = Queue(id=2, num_servers=1, capacity=5, service_time_range=(2,3))
        # Schedule first arrival to queue 1
        self.schedule_event(Event(time=self.time, event_type='arrival', queue_id=1))
        # Schedule next arrival to queue 1
        self.schedule_next_arrival_to_queue1()

    def schedule_event(self, event):
        heapq.heappush(self.event_list, event)
    
    def schedule_next_arrival_to_queue1(self):
        # Generate next inter-arrival time for queue 1
        inter_arrival_time = random.uniform(1, 4)
        self.random_numbers_used += 1
        arrival_time = self.time + inter_arrival_time
        event = Event(time=arrival_time, event_type='arrival', queue_id=1)
        self.schedule_event(event)
    
    def run(self):
        while self.event_list and self.random_numbers_used < 100000:
            event = heapq.heappop(self.event_list)
            self.time = event.time
            self.handle_event(event)
        # Update state times for each queue to account for time after last event
        for queue in self.queues.values():
            state = queue.num_in_queue
            time_in_state = self.time - queue.last_event_time
            if state in queue.state_times:
                queue.state_times[state] += time_in_state
            else:
                queue.state_times[state] = time_in_state
            queue.last_event_time = self.time
        # Simulation ends
        self.report_statistics()
    
    def handle_event(self, event):
        if event.event_type == 'arrival':
            self.handle_arrival(event)
        elif event.event_type == 'departure':
            self.handle_departure(event)
        else:
            pass  # Unknown event type

    def handle_arrival(self, event):
        queue = self.queues[event.queue_id]
        # Update state times
        self.update_queue_state_time(queue)
        if queue.num_in_queue < queue.capacity:
            # Customer enters the queue
            queue.num_in_queue += 1
            customer_id = self.customer_id_counter
            self.customer_id_counter +=1
            customer = Customer(customer_id)
            self.customers[customer_id] = customer
            # If server is available, start service
            if queue.num_in_service < queue.num_servers:
                queue.num_in_service +=1
                self.start_service(queue, customer_id)
            else:
                # Customer waits in queue
                queue.waiting_line.append(customer_id)
            # For arrivals to queue 1 from outside, schedule next arrival
            if event.queue_id == 1:
                self.schedule_next_arrival_to_queue1()
        else:
            # Queue is full, customer is lost
            queue.lost_customers +=1
            # For arrivals to queue 1 from outside, schedule next arrival
            if event.queue_id == 1:
                self.schedule_next_arrival_to_queue1()
    
    def start_service(self, queue, customer_id):
        # Generate service time
        min_service, max_service = queue.service_time_range
        service_time = random.uniform(min_service, max_service)
        self.random_numbers_used +=1
        departure_time = self.time + service_time
        event = Event(time=departure_time, event_type='departure', queue_id=queue.id, customer_id=customer_id)
        self.schedule_event(event)
    
    def handle_departure(self, event):
        queue = self.queues[event.queue_id]
        # Update state times
        self.update_queue_state_time(queue)
        queue.num_in_queue -=1
        queue.num_in_service -=1
        customer_id = event.customer_id
        # Remove customer from system
        del self.customers[customer_id]
        if queue.id == 1:
            # Customer goes to queue 2
            arrival_event = Event(time=self.time, event_type='arrival', queue_id=2)
            self.schedule_event(arrival_event)
        # If there are customers waiting, start service for the next one
        if queue.waiting_line:
            next_customer_id = queue.waiting_line.popleft()
            queue.num_in_service +=1
            self.start_service(queue, next_customer_id)
    
    def update_queue_state_time(self, queue):
        state = queue.num_in_queue
        time_in_state = self.time - queue.last_event_time
        if state in queue.state_times:
            queue.state_times[state] += time_in_state
        else:
            queue.state_times[state] = time_in_state
        queue.last_event_time = self.time
    
    def report_statistics(self):
        # Report for each queue
        total_time = self.time - 1.5  # Since simulation starts at time 1.5
        print(f"Total simulation time: {total_time}")
        for queue_id, queue in self.queues.items():
            print(f"\nQueue {queue_id} Statistics:")
            print(f"Lost customers: {queue.lost_customers}")
            print("State probabilities:")
            for state in sorted(queue.state_times.keys()):
                time_in_state = queue.state_times[state]
                probability = time_in_state / total_time
                print(f"  State {state}: {probability:.6f}")
            print("Accumulated times in states:")
            for state in sorted(queue.state_times.keys()):
                time_in_state = queue.state_times[state]
                print(f"  State {state}: {time_in_state:.4f}")


In [3]:

if __name__ == "__main__":
    sim = Simulation()
    sim.run()


Total simulation time: 56994.83348665569

Queue 1 Statistics:
Lost customers: 14018
State probabilities:
  State 0: 0.000052
  State 1: 0.059498
  State 2: 0.449026
  State 3: 0.491423
Accumulated times in states:
  State 0: 2.9609
  State 1: 3391.0995
  State 2: 25592.1879
  State 3: 28008.5852

Queue 2 Statistics:
Lost customers: 8812
State probabilities:
  State 0: 0.000058
  State 1: 0.000107
  State 2: 0.000111
  State 3: 0.008413
  State 4: 0.431367
  State 5: 0.559944
Accumulated times in states:
  State 0: 3.3134
  State 1: 6.1089
  State 2: 6.3272
  State 3: 479.4848
  State 4: 24585.7053
  State 5: 31913.8938
